<a href="https://colab.research.google.com/github/Jason1Borne/ML_practice/blob/master/%E8%87%AA%E5%B7%B1%E4%BF%AE%E6%94%B9%E4%BA%BA%E8%84%B8%E8%AF%86%E5%88%ABbaseline%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 构建基础baseline模型

---

需要使用的工具
- face_recognition
- cv2
- PIL
- tensorflow
- keras
- gc 
- glob

`pip install face_recognition`

`pip install pillow`

`pip install glob`

## 一、截取人脸


***截取人脸***

<img src="face_img.jpg" style="zoom:20%" >

<img src="face_img.png" style="zoom:50%" >

1. 截取人脸（注意一幅图多人脸情况）
2. 格式转换
3. 图片放缩（注意等比）
4. 保存人脸

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 1.3MB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566176 sha256=30624ff109bcbc95df1477bfc0495358ebe4609ce7534b23ce3ae4bc6eadc1b0
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [0]:
import face_recognition
import glob
import cv2
from PIL import Image
import os
import pickle
import gc
import numpy as np
import tensorflow as tf

class preprocess(object):
    def __init__(self,scale_size=139):
        self.img_cant = []#不能被截取人脸的图片路径
        self.scale_size = scale_size#缩放大小的比例

    @staticmethod
    def show_face(path):
        image=face_recognition.load_image_file(path)
        face_locations=face_recognition.face_locations(image)#一张图片可能有多张人脸所以是个list
        if len(face_locations)==0:
            return -1
        for k,i in enumerate(face_locations):#i 就是生成的4个坐标可以框出一个人脸
            (a,b,c,d)=i
            image_spilt=image[a:c,d:b,:]
            cv2.imshow('img_{}'.format(k),image_spilt)
            return 1


    def split_face(self,in_path,out_path,use_cnn):
        image=face_recognition.load_image_file(in_path)#导入图片
        if max(image.shape)>2000:
            if image.shape[0]>image.shape[1]:
                #等比放缩，避免畸变
                image=cv2.resize(image,(2000,int(2000*image.shape[1]/image.shape[0])))
            else:
                image=cv2.resize(image,(int(2000*image.shape[0]/image.shape[1]),2000))

        #使用cnn和不适用cnn来找人脸
        if not use_cnn:
            face_locations=face_recognition.face_locations(image)
        else:
            face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=1, model="cnn")
        #取的这张图片的地址名字作为名字
        img_name=os.path.basename(in_path)
        
        #没有找到人脸就进行提示
        if len(face_locations)==0:
            self.img_cant.append(img_name)
            print(img_name)
            return
        for k,i in enumerate(face_locations):
            (a,b,c,d)=i
            image_spilt=image[a:c,d:b,:]
            #进行放缩
            image_spilt = self.scale_img(image_spilt)
            img=Image.fromarray(image_spilt)
            img.save(out_path+'/{}_{}.png'.format(img_name,k))
            print('success')

    def scale_img(self,img):
        h, w = img.shape[:2]
        #等比放缩
        if h > w:
            new_h, new_w = self.scale_size * h / w, self.scale_size
        else:
            new_h, new_w = self.scale_size, self.scale_size * w / h
        
        new_h, new_w = int(new_h), int(new_w)
        img = cv2.resize(img, (new_w, new_h))
        
        if new_h == new_w:
            return img 
        elif new_h < new_w:
            top = 0
            left = np.random.randint(0, new_w - self.scale_size)
            #随机截断
        elif new_h > new_w:
            
            top = np.random.randint(0, new_h - self.scale_size)
            left = 0
            
                
        img = img[top: top + self.scale_size,
                  left: left + self.scale_size]
        return img

    def preprocess(self,in_path,out_path,use_cnn=False):
        path_lst=glob.glob(in_path+'/*.jpg')
        for index,path in enumerate(path_lst):
            if index % 20==0:
                gc.collect()
                #每隔20清一次
            print(path)
            self.split_face(path,out_path,use_cnn)

In [0]:
dir_lst = ['/content/drive/My Drive/Colab Notebooks/表情识别/{}'.format(i) for i in range(5)]
train_ins = preprocess()
for in_path in dir_lst:
    out_path = '/content/drive/My Drive/Colab Notebooks/表情识别/'+ os.path.basename(in_path)+'_face'
    if not os.path.exists(out_path):
        os.mkdir(out_path)
    train_ins.preprocess(in_path,out_path,use_cnn=False)
    with open('train_face_cant_{}.pkl'.format(os.path.basename(in_path)),'wb') as f:
        pickle.dump(train_ins.img_cant,f)

---
# 二、生成tfrecord

In [0]:
#图像标签生成tfrecord函数
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
from PIL import Image
import numpy as np
import sys
def image2tfrecord(image_list,label_list,filename):
    '''
    image_list:image path list
    label_list:label list
    '''
    length=len(image_list)
    writer=tf.python_io.TFRecordWriter(filename)
    for i in range(length):
        if i % 100==0:
            ratio=round(i/float(length),4)
            sys.stdout.write('ratio:{}\r'.format(ratio))
            sys.stdout.flush()
        image=Image.open(image_list[i])
        if 'png' in image_list[i][-4:]:
            if image.mode=='RGB':
                r, g, b = image.split()
                image = Image.merge("RGB", (r, g, b))
            elif image.mode=='L':
                pass
            else:
                r,g, b, a = image.split()
                image = Image.merge("RGB", (r, g, b))
        image=image.resize((139,139))
        #这个地方就展开了
        image_bytes=image.tobytes()
        features={}
        features['image']=tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))
        features['label']=tf.train.Feature(int64_list=tf.train.Int64List(value=[int(label_list[i])]))
        tf_features=tf.train.Features(feature=features)
        tf_example=tf.train.Example(features=tf_features)
        tf_serialized=tf_example.SerializeToString()
        writer.write(tf_serialized)
    writer.close()

In [0]:
cwd = os.path.join(os.getcwd(),'face')
# 生成10个tfrecords
# 划分了训练集和验证集
random_ratio = 0.05
sum_number = 0
for i in range(5):
    print(i)
    image_list = np.array(glob.glob(os.path.join(cwd,'{}_face'.format(i))+'/*.png'))
    test_choice = np.random.choice(range(len(image_list)),size=int(len(image_list)*random_ratio))
    train_choice = list(set(range(len(image_list))).difference(test_choice))
    image_list_train = image_list[train_choice]
    image_list_test = image_list[test_choice]
    label_list = np.zeros(len(image_list))+i
    image2tfrecord(image_list_train,np.zeros(len(image_list_train))+i,'face_train_{}.tfrecords'.format(i))
    image2tfrecord(image_list_test,np.zeros(len(image_list_test))+i,'face_test_{}.tfrecords'.format(i))

0
1atio:0.0119
2atio:0.9333
3atio:0.9479
4atio:0.0485


---
# 三、使用生成器准备数据

In [0]:
#解析函数
def pares_tf(example_proto):
    dics={}
    dics['label']=tf.FixedLenFeature((),dtype=tf.int64,default_value=0)
    dics['image']=tf.FixedLenFeature((),dtype=tf.string,default_value="")

    parsed_example=tf.parse_single_example(serialized=example_proto,features=dics)
    image=tf.decode_raw(parsed_example['image'],out_type=tf.uint8)
    #image=tf.image.decode_jpeg(parsed_example['image'], channels=1)
    #这个地方可以加一些操作
    image=tf.reshape(image,(139,139,3))
    
    #image = tf.image.rgb_to_grayscale(image)
    image=pre_process(image)
    
    image=tf.cast(image,tf.float32)/255
    #标签的操作
    label=parsed_example['label']
    label=tf.cast(label,tf.int32)
    label = tf.one_hot(label,depth=5,on_value=1.0,off_value=0.0)
    return image,label

In [0]:
#dataset
def dataset(filenames,batch_size,epochs):
    dataset=tf.data.TFRecordDataset(filenames=filenames)
    new_dataset=dataset.map(pares_tf)
    
    return new_dataset

In [0]:
# 图像预处理
def pre_process(images,random_flip_up_down=False,random_flip_left_right=False,random_brightness=True,random_contrast=True,random_saturation=False,random_hue=False):
    if random_flip_up_down:
        images = tf.image.random_flip_up_down(images)
    if random_flip_left_right:
        images = tf.image.random_flip_left_right(images)
    if random_brightness:
        images = tf.image.random_brightness(images, max_delta=0.2)
    if random_contrast:
        images = tf.image.random_contrast(images, 0.9, 1.1)
    if random_saturation:
        images = tf.image.random_saturation(images, 0.3, 0.5)
    if random_hue:
        images = tf.image.random_hue(images,0.2)
#     new_size = tf.constant([160,160],dtype=tf.int32)
#     images = tf.image.resize_images(images, new_size)
    return images

In [0]:
tf.reset_default_graph()
filenames=['/content/drive/My Drive/Colab Notebooks/表情识别/face_train_{}.tfrecords'.format(i) for i in range(5)]
next_element=dataset(filenames,batch_size=5,epochs=1)
init = tf.global_variables_initializer()

In [0]:
with tf.Session() as sess:
    sess.run(init)
    batch_images,batch_labels=sess.run([next_element[0],next_element[1]])
    for i in range(batch_images.shape[0]):
        print(batch_images[i].shape)
        img = np.array(batch_images[i])

TypeError: ignored

---
# 四、构建模型

In [0]:
import tensorflow as tf
import sys
import glob
from tensorflow.keras.models import Sequential, Model

In [0]:
tf.reset_default_graph()

In [0]:
from tensorflow.keras.applications.vgg19 import VGG19

input_data = tf.keras.Input(shape=(139,139,3))

inception_model = VGG19(include_top=False, weights='imagenet', input_tensor=input_data, input_shape=(139, 139, 3))

inception_model.trainable = False

# inception_model.layers[1].trainable = True

# inception_model.layers[4].trainable = True

output = inception_model.output

output = tf.keras.layers.Conv2D(512, (3,3), padding='same', strides=1, activation='relu')(output)


output = tf.keras.layers.Conv2D(1024, (2,2), padding='same', strides=1, activation='relu')(output)

output = tf.keras.layers.Flatten()(output)

output = tf.keras.layers.Dense(128, activation='relu')(output)

output = tf.keras.layers.Dense(64, activation='relu')(output)

output = tf.keras.layers.Dropout(0.5)(output)

output = tf.keras.layers.Dense(5, activation='softmax')(output)

model = Model(input_data, output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
80142336/80134624 [==============================] - 1s 0us/step


In [0]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 139, 139, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 139, 139, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 139, 139, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 69, 69, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 69, 69, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 69, 69, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 34, 34, 128)       0     

In [0]:
# input_data = tf.placeholder(tf.float32,shape=(None,139,139,3))
# input_label = tf.placeholder(tf.float32,shape=(None,5))
input_data = tf.keras.Input(shape=(139,139,1))
#139
hidden = tf.keras.layers.Conv2D(filters=16,kernel_size=3,strides=1,padding='valid',activation='relu')(input_data)
#137
hidden = tf.keras.layers.MaxPool2D(pool_size=2)(hidden)
#68
hidden = tf.keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding='same',activation='relu')(hidden)
#34
hidden = tf.keras.layers.MaxPool2D(pool_size=2)(hidden)
#17
hidden = tf.keras.layers.Conv2D(filters=64,kernel_size=3,strides=2,padding='valid',activation='relu')(hidden)
#8
hidden = tf.keras.layers.MaxPool2D(pool_size=2)(hidden)
#4
hidden = tf.keras.layers.Conv2D(filters=128,kernel_size=3,strides=2,padding='valid',activation='relu')(hidden)
#1
hidden = tf.keras.layers.Flatten()(hidden)

hidden = tf.keras.layers.Dense(32, activation='relu')(hidden)

output = tf.keras.layers.Dense(5,activation='softmax')(hidden)

model = Model(input_data, output)




model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])





In [0]:
from tensorflow.keras.models import load_model
model = load_model('/content/drive/My Drive/Colab Notebooks/facenet_keras.h5')

In [0]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/facenet_keras_weights.h5')

In [0]:
model.trainable = False

In [0]:
input_data = tf.keras.Input(shape=(160,160,3))

hidden = model(input_data)


output = tf.keras.layers.Dense(128,activation='relu')(hidden)

output = tf.keras.layers.Dense(64,activation='relu')(hidden)

output = tf.keras.layers.Dense(32,activation='relu')(hidden)

output = tf.keras.layers.Dense(5,activation='softmax')(hidden)

model = Model(input_data, output)


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])


In [0]:
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [0]:
test_num = sum([int(len(glob.glob('/content/drive/My Drive/Colab Notebooks/表情识别/{}_face/*.png'.format(i)))*random_ratio) for i in range(5)])
train_num = sum([len(glob.glob('/content/drive/My Drive/Colab Notebooks/表情识别/{}_face/*.png'.format(i))) for i in range(5)])-test_num


NameError: ignored

In [0]:
for layer in model.layers:
    print(layer.trainable)

True
False
True
True


In [0]:


steps_per_epoch = 1000
batch_size = 128
total_sum = 0
epochs = 50
random_ratio = 0.05

filenames_train=['/content/drive/My Drive/Colab Notebooks/表情识别/face_train_{}.tfrecords'.format(i) for i in range(5)]
filenames_test=['/content/drive/My Drive/Colab Notebooks/表情识别/face_test_{}.tfrecords'.format(i) for i in range(5)]

# test_num = sum([int(len(glob.glob('./face/{}_face/*.png'.format(i)))*random_ratio) for i in range(5)])
# train_num = sum([len(glob.glob('./face/{}_face/*.png'.format(i))) for i in range(5)])-test_num



train_dataset=dataset(filenames_train,batch_size=batch_size,epochs=epochs)
train_dataset=train_dataset.shuffle(buffer_size=(1000000))
train_dataset=train_dataset.batch(batch_size).repeat(epochs*steps_per_epoch)
train_dataset=train_dataset.prefetch(1)


test_dataset=dataset(filenames_test,batch_size=batch_size,epochs=1)
test_dataset=test_dataset.shuffle(buffer_size=(100000))
test_dataset=test_dataset.batch(batch_size).repeat(20)
test_dataset=test_dataset.prefetch(1)


In [0]:
model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=epochs,validation_data=test_dataset,validation_steps=20, verbose=1)#)

Train on 1000 steps, validate on 20 steps
Epoch 1/50
 898/1000 [=========================>....] - ETA: 1:20 - loss: 0.7126 - acc: 0.7506

In [0]:
# tf.add_to_collection('my_op',input_data)
# tf.add_to_collection('my_op',output)
# tf.add_to_collection('my_op',loss)

# init = tf.global_variables_initializer()
# saver = tf.train.Saver()
# with tf.Session() as sess:
#     sess.run([init])
#     while epoch<epochs:
#         data,label=sess.run([next_element_train[0],next_element_train[1]])
#         total_sum+=batch_size
#         _,loss_val = sess.run([train_op,loss],feed_dict={input_data:data,input_label:label})
#          if total_sum%20==0:
#             sys.stdout.write('epoch:{},index:{}\\{},loss:{:.4f}\r'.format(epoch,total_sum%train_num,train_num,loss_val))
#             sys.stdout.flush()
#         if total_sum//train_num>epoch:
#             epoch = total_sum//train_num
#             loss_val = sess.run([loss],feed_dict={input_data:data,input_label:label})
            
#             next_element_test=dataset(filenames_test,batch_size=batch_size,epochs=1)
#             total_test_sum = 0
#             acc_test_lst = []
#             while total_test_sum<test_num:
#                 total_test_sum+=batch_size
#                 test_data,test_label=sess.run([next_element_test[0],next_element_test[1]])
#                 acc_test = sess.run([acc],feed_dict={input_data:test_data,input_label:test_label})
#                 acc_test_lst.append(acc_test[0])
#             acc_val = sum(acc_test_lst)/len(acc_test_lst)
#             saver.save(sess, save_path="./model/my_model.ckpt")
#             print('epoch:{},train_loss:{:.4f},test_acc:{:.4f}'.format(epoch,loss_val[0],acc_val))